In [ ]:
import os
import json
from skyfield.sgp4lib import EarthSatellite
from skyfield.api import load, Topos
from datetime import datetime, timedelta
import math
import time
from haversine import haversine


In [ ]:
start_time_str = "2023-10-03 00:00:00"
end_time_str = "2023-10-04 00:00:00"

# Convert the input strings to datetime objects
start_time = datetime.strptime(start_time_str, "%Y-%m-%d %H:%M:%S")
end_time = datetime.strptime(end_time_str, "%Y-%m-%d %H:%M:%S")


In [ ]:
class ground_station:

    def __init__(self, name, lat, long, height, mask_receive,
                 mask_transmit, uplink_rate, downlink_rate):

        self.Name = name

        self.Latitude = lat
        self.Longitude = long
        self.Height = height

        self.StationMaskRecv = mask_receive
        self.StationMaskTrans = mask_transmit
        self.UplinkRate = uplink_rate    
        self.DownlinkRate = downlink_rate

        self.RecofigTime = 5*60
        
        self.topos = Topos(lat, long)
        
        self.availableSlots = []
        self.allocatedSlots = []
        self.start = None
        self.end = None
        
    def isVisibleRecv(self, sat, tm): 
        relative_pos =  (sat.satObj - self.topos).at(tm)
        elevation_angle = relative_pos.altaz()[0]
        
        if elevation_angle.degrees > self.StationMaskRecv:
            return True
        
        return False

    def isVisibleTrans(self, sat, tm): 
        relative_pos =  (sat.satObj - self.topos).at(tm)
        elevation_angle = relative_pos.altaz()[0]
        
        if elevation_angle.degrees > self.StationMaskTrans:
            return True
        
        return False
    
    def reset(self, start, end):
        if len(self.availableSlots) == 0:
            slot = {}
            slot["Start"] = start
            slot["End"] = end
        
            self.availableSlots = [slot]
            self.allocatedSlots = []
            
            self.start = start
            self.end = end            
        else:
            startSlot = self.availableSlots[0]
            endSlot = self.availableSlots[-1]
            
            for slot in self.availableSlots:
                if slot["Start"].tt < startSlot["Start"].tt:
                    startSlot = slot

                if slot["End"].tt > endSlot["End"].tt:
                    endSlot = slot
                    
            if start.tt < self.start.tt:   
                if (self.start.tt - startSlot["Start"].tt)  > 0.5/(3600*24):
                    slot = {}
                    slot["Start"] = start
                    slot["End"] = self.start

                    self.availableSlots.append(slot)
                    self.start = start
                elif abs(self.start.tt - startSlot["Start"].tt)  < 0.5/(3600*24):
                    startSlot["Start"] = start
                    self.start = start
            
            if end.tt > self.end.tt:
                if (endSlot["End"].tt - self.end.tt)  > 0.5/(3600*24):
                    slot = {}
                    slot["Start"] = self.end
                    slot["End"] = end

                    self.availableSlots.append(slot)
                    self.end = end
                elif abs(endSlot["End"].tt - self.end.tt)  < 0.5/(3600*24):
                    endSlot["End"] = end
                    self.end = end
                
                    
    def strToTm(self, ts, Str):        
        dt = datetime.strptime(Str, "%Y-%m-%dT%H:%M:%S")
        return ts.utc(dt.year, dt.month, dt.day, dt.hour, dt.minute, dt.second)

    def canUplink(self, sched, start, ts, visibility):
        transfer_dur = len(json.dumps(sched)) / (self.UplinkRate/8)        
        delivery = self.strToTm(ts, sched['Activity Window']["Start"])
        
        return self.getSlot(start, delivery, transfer_dur, visibility)
        
    def canDownlink(self, ev, tm, ts, visibility):

        duration = ev["Ref"]['Transfer Time']
        start = ts.utc(tm.utc_datetime() + timedelta(seconds=duration))

        transfer_dur = ev["Ref"]['Storage'] / (self.DownlinkRate/8)
        
        delivery = self.strToTm(ts, ev["Ref"]['DeliveryTime'])
        
        return self.getSlot(start, delivery, transfer_dur, visibility)
    
    def getSlot(self, req_start, req_end, duration, visibility):
        
        if req_end.tt < req_start.tt +duration/(3600.0*24):
            return [False, req_start, {}]
                    
        for slot in self.availableSlots:
            
            for window in visibility:

                slot_start = slot["Start"]
                slot_end = slot["End"]

                if req_start.tt > window["Start"].tt:
                    start = req_start
                else:
                    start = window["Start"]

                if slot_start.tt > start.tt:
                    start = slot_start
                    
                if req_end.tt < window["End"].tt:
                    end = req_end
                else:
                    end = window["End"]

                if slot_end.tt < end.tt:
                    end = slot_end
                    
                if end.tt > start.tt + duration/(3600.0*24):
                    #print("Res", start.utc_strftime(), end.utc_strftime())
                    return [True, start, slot]
                
        return [False, req_start, {}]
    
    def bookSlot(self, ev, start, slot, ts):
        
        duration = ev["Ref"]['Storage'] / self.DownlinkRate
        
        #print("bookSlot", start.utc_strftime(), slot["Start"].utc_strftime(), slot["End"].utc_strftime(), duration)
        #print(slot['Start'].utc_strftime(), slot['End'].utc_strftime())
        
        if start.tt > slot["Start"].tt:
        
            slotA = {}
            slotA["Start"] = slot["Start"]
            slotA["End"] =   start
            
            self.availableSlots.append(slotA)
            #print("Slot A", slotA['Start'].utc_strftime(), slotA['End'].utc_strftime())

        if start.tt + (duration+5)/(3600.0*24) < slot["End"].tt:
        
            slotB = {}
            slotB["Start"] = ts.utc(start.utc_datetime() + timedelta(seconds=duration))
            slotB["End"] =   slot["End"]
            
            self.availableSlots.append(slotB)
            #print("Slot B", slotB['Start'].utc_strftime(), slotB['End'].utc_strftime())

        slotN = {}
        slotN["Start"] = start
        slotN["End"] =   ts.utc(start.utc_datetime() + timedelta(seconds=duration))
        
        #print("Slot Allocated", slotN['Start'].utc_strftime(), slotN['End'].utc_strftime())

        ev["Ref"]["Slot"] = slotN
        ev["Ref"]["Station"] = self.Name

        self.allocatedSlots.append(slotN)

        self.availableSlots.remove(slot)

        return slotN
        


In [ ]:
class satelite:

    def __init__(self, tle):

        self.loadTLE(tle)
        self.initPower()
        self.StorageCapacityMax = 40*1024*1024*1024
        self.StorageCapacity = 40*1024*1024*1024
        self.viewAngle = 30
        self.viewRatio = math.sin(self.viewAngle*math.pi/180)
        self.lat = -100000
        self.long = -100000
        self.maint = []
        self.Events = []
        self.Scheduled = []
        self.outages = []
        self.nightSlot = {}
        self.nightSlot["Start"] = None
        self.nightSlot["Slots"] = []
        
        self.stationVisibility = {}
        self.cacheDict = {}


    def loadTLE(self, tle):

        ts = load.timescale() # Create timescale object for TLE computation

        with open(tle) as f: # For-loop and f-string used to open the TLE files for SOSO-1, SOSO-2, etc.
            data = json.load(f) # Load the JSON data from the file
            self.name = data['name']
            line1 = data['line1']
            line2 = data['line2']
        self.satObj = EarthSatellite(line1, line2, self.name, ts) # Create new satellite object where line 1 = tle[1], line 2 = tle[2], title = tle[0], and ts for timescale

    def initPower(self):
        # Power Management Example
        self.P_sunlit = 500 # in Watts during Sunlight        
        # 200-800 Watts for research sat.
        # 1000-1500 Watts for commercial sat.
        self.P_eclipse = self.P_sunlit * 0.4 # in Watts during Eclipse (assuming 40% of power is used)
        self.nightLimitMax = 6*60

    def update(self, tm, eph, images, ts, GroundStations):
        
        timeS = int(tm.tt*60*24)*60
        if timeS not in self.cacheDict:
            
            self.cacheDict[timeS] = {}
            self.cacheDict[timeS]["pos"] = self.satObj.at(tm).position.km  # Plain (x, y, z) coordinates at the current time (Center of Earth)
            subpnt = self.satObj.at(tm).subpoint()

            self.cacheDict[timeS]["lat"] = subpnt.latitude.degrees  # Latitude at the current time
            self.cacheDict[timeS]["long"] = subpnt.longitude.degrees  # Longitude at the current time
            
            # Calculate altitude from position data
            semi_major_axis_km = self.satObj.model.a * 6378.137  # Get the semi-major axis in kilometers
            altitude = semi_major_axis_km - 6378.137  # The altitude is the semi-major axis minus the Earth's radius
            self.cacheDict[timeS]["fov"] = self.viewRatio * altitude

            
            is_sunlit = self.satObj.at(tm).is_sunlit(eph) # Check if satellite is sunlit at current time
            self.processDayNight(tm, is_sunlit)
            
        pos = self.cacheDict[timeS]["pos"]  # Plain (x, y, z) coordinates at the current time (Center of Earth)
        
        pLat = self.lat
        pLong = self.long
        
        self.lat = self.cacheDict[timeS]["lat"]  # Latitude at the current time
        self.long = self.cacheDict[timeS]["long"]  # Longitude at the current time
        self.fov = self.cacheDict[timeS]["fov"]
        
        #Dec = 30.0
        Dec = 2.0

        dLat = self.lat - pLat
        dLong = self.long - pLong

        dLat = dLat / Dec
        dLong = dLong / Dec

        if abs(self.lat - pLat) < 0.00001:
            return

        if abs(self.long - pLong) < 0.00001:
            return

        if pLat == -100000:
            return

        if pLong == -100000:
            return

        Lat = pLat
        Long = pLong
        for i in range(int(Dec)):    

            if Lat < -90:
                Lat = Lat + 180
                
            if Lat > 90:
                Lat = Lat - 180

            if Long < -180:
                Long = Long + 360
                
            if Long > 180:
                Long = Long - 360

            self.process_images(images, Lat, Long, tm)
            self.process_ground_stations(tm, GroundStations)
                        
            Lat = (Lat + dLat)
            Long = Long + dLong
            tm = ts.utc(tm.utc_datetime() + timedelta(seconds=60/Dec))

    def processDayNight(self, tm, is_sunlit):
                
        if not is_sunlit:
            if self.nightSlot["Start"] is None:
                self.nightSlot["Start"] = tm
        else:
            if self.nightSlot["Start"] is not None:
                slot = {}
                slot["Start"] = self.nightSlot["Start"]
                slot["End"] = tm
                slot["Total Time"] = 0
                self.nightSlot["Slots"].append(slot)
                self.nightSlot["Start"] = None
                        
    def process_ground_stations(self, tm, stations):
        for station in stations:
            if station.Name not in self.stationVisibility:
                self.stationVisibility[station.Name] = {}
                self.stationVisibility[station.Name]["Start"] = None
                self.stationVisibility[station.Name]["slots"] = []
            
            if station.isVisibleTrans(self, tm):
                if self.stationVisibility[station.Name]["Start"] is None:
                    self.stationVisibility[station.Name]["Start"] = tm
            else:
                if self.stationVisibility[station.Name]["Start"] is not None:

                    slot = {}
                    slot["Start"] = self.stationVisibility[station.Name]["Start"]
                    slot["End"] = tm
                    
                    
                    #if slot["Start"].tt > slot["End"].tt:
                    #    print("Error")
                    #else:
                    if slot["Start"].tt <= slot["End"].tt:
                        self.stationVisibility[station.Name]["slots"].append(slot)
                    
                    self.stationVisibility[station.Name]["Start"] = None
                
                
            
    def intersect(self, image, Lat, Long):

        distX = haversine((image['Latitude'], Long), (Lat, Long))
        distY = haversine((Lat, image['Longitude']), (Lat, Long))
        
        L = 0.5*image['Length']
        W = 0.5*image['Width']

        if distX < (self.fov - L) and distY < (self.fov - W):
            return True
        else:
            return False

    def logImageStart(self, im, tm):

        if self.name not in im["Sources"]:
            im["Sources"][self.name] = {}            
            im["Sources"][self.name]["session start"] = tm
            im["Sources"][self.name]["sessions"] = []
            return

        if im["Sources"][self.name]["session start"] is None:
            im["Sources"][self.name]["session start"] = tm
            return

    def logImageEnd(self, im, tm):

        if self.name not in im["Sources"]:
            return

        if im["Sources"][self.name]["session start"] is None:
            return

        dT = tm - im["Sources"][self.name]["session start"]
        dT = dT * 24 * 3600

        if dT > im['Transfer Time']:
            im["Sources"][self.name]["sessions"].append((im["Sources"][self.name]["session start"], tm, dT))

        im["Sources"][self.name]["session start"] = None
            
    def process_images(self, images, Lat, Long, tm):

        ind = 1
        for im in images:

            if im["Processed"]:
            #if im["Completed"]:
                
                continue
            
            if tm.tt < im["ImageStartTimeObj"].tt:
                continue

            if tm.tt > im["ImageEndTimeObj"].tt:
                continue

            if self.intersect(im, Lat, Long):
                self.logImageStart(im, tm)
            else:
                self.logImageEnd(im, tm)
                                
            ind = ind +1
            
        return

    def process_images_final(self, images, tm):
        for im in images:

            if im["Processed"]:
            #if im["Completed"]:
                continue

            self.logImageEnd(im, tm)

    def comp_time_str(self, t1_str, t2_str):

        t1 = datetime.strptime(t1_str, "%Y-%m-%dT%H:%M:%S")
        t2 = datetime.strptime(t2_str, "%Y-%m-%dT%H:%M:%S")

        if t1 > t2:
            return 1
        
        if t1 == t2:
            return 0

        if t1 < t2:
            return 1

    def add_children(self, ev, tm):

        if ev["Type"] == "Image Order":
            pass
        elif ev["Type"] == "Maintainence":
            pass
        else:
            print("Unknown event")

    def register_event(self, tm, ev, ts):

        #print("Registered Event")

        ev_begin = tm
        ev_dur = ev["Ref"]['Transfer Time']
        ev_end = ts.utc(tm.utc_datetime() + timedelta(seconds=ev_dur))

        self.Scheduled.append((ev_begin, ev_end, ev))

        ev["Ref"]["Completed"] = True
        self.add_children(ev, tm)
        ev["Ref"]["Time Start"] = tm
        ev["Ref"]["Time End"] = ev_end
        ev["Ref"]["Sat"] = self.name
        
        self.Events.remove(ev)
        
        self.StorageCapacity = self.StorageCapacity - ev["Ref"]['Storage']
        
        for slot in self.nightSlot["Slots"]:
            ev_begin = slot["Start"]
            ev_end = slot["End"]
            
            if ev["Start"].tt > ev_begin.tt and ev["Start"].tt < ev_end.tt:
                slot["Total Time"] = slot["Total Time"] + ev_dur
                if slot["Total Time"] > self.nightLimitMax:
                    print("Error: Night Limit exceeded to", slot["Total Time"], ", Slot", slot)

            elif ev["End"].tt > ev_begin.tt and ev["End"].tt < ev_end.tt:
                slot["Total Time"] = slot["Total Time"] + ev_dur
                if slot["Total Time"] > self.nightLimitMax:
                    print("Error: Night Limit exceeded to", slot["Total Time"], ", Slot", slot)

            elif ev_begin.tt > ev["Start"].tt and ev_begin.tt < ev["End"].tt:
                slot["Total Time"] = slot["Total Time"] + ev_dur
                if slot["Total Time"] > self.nightLimitMax:
                    print("Error: Night Limit exceeded to", slot["Total Time"], ", Slot", slot)

            elif ev_end.tt > ev["Start"].tt and ev_end.tt < ev["End"].tt:
                slot["Total Time"] = slot["Total Time"] + ev_dur
                if slot["Total Time"] > self.nightLimitMax:
                    print("Error: Night Limit exceeded to", slot["Total Time"], ", Slot", slot)

    def strToTm(self, ts, Str):        
        dt = datetime.strptime(Str, "%Y-%m-%dT%H:%M:%S")
        return ts.utc(dt.year, dt.month, dt.day, dt.hour, dt.minute, dt.second)
        
    def add_maint_event(self, ts, ev0):
        
        if ev0["Ref"]["RepeatCycle"]['Repetition'] == 'Null':
            return
        
        parent = ev0
        
        dur = int(ev0["Ref"]["Duration"])
        minGap = int(ev0["Ref"]["RepeatCycle"]["Frequency"]["MinimumGap"])
        maxGap = int(ev0["Ref"]["RepeatCycle"]["Frequency"]["MaximumGap"])
        
        for i in range(ev0["Level"]+1, int(ev0["Ref"]["RepeatCycle"]["Repetition"])):

            ev = {}
            ev["Type"] = "Maintainence"
            
            ev["Start"] = ts.utc(parent["Start"].utc_datetime() + timedelta(seconds=dur+minGap))
            ev["End"] = ts.utc(parent["End"].utc_datetime() + timedelta(seconds=maxGap))
            
            ev["Ref"] = parent["Ref"]
            ev["Parent"] = parent            

            parent["Child"].append(ev)
            ev["Child"] = []
            
            ev["Level"] = i
            
            self.Events.append(ev)
            
            parent = ev

    def bifurcate_maint(self, ev, ev_begin, ev_end, ts):
        # print("Splitting ", ev["Ref"]['Activity'], "(", ev["Start"].tt, ev["End"].tt, ') by (', ev_begin.tt, ev_end.tt, ")")
        
        if (ev_begin - ev["Start"]) > int(ev["Ref"]['Duration'])/(24.0*3600.0):
            evA = {}
            evA["Type"] = "Maintainence"
            evA["Start"] = ev["Start"]
            evA["End"] = ts.utc(ev_begin.utc_datetime() - timedelta(seconds=5))
            evA["Ref"] = ev["Ref"]
            evA["Parent"] = ev["Parent"]            
            evA["Child"] = []
            evA["Level"] = ev["Level"]
            
            self.Events.append(evA)
            # print("    NodeA", evA["Start"].tt, evA["End"].tt)

            if ev["Parent"] is not None:
                ev["Parent"]["Child"].append(evA)

            self.add_maint_event(ts, evA)
            

        if (ev["End"] - ev_end) > int(ev["Ref"]['Duration'])/(24.0*3600.0):
            evB = {}
            evB["Type"] = "Maintainence"
            evB["Start"] = ts.utc(ev_end.utc_datetime() + timedelta(seconds=5))
            evB["End"] = ev["End"]
            evB["Ref"] = ev["Ref"]
            evB["Parent"] = ev["Parent"]            
            evB["Child"] = []
            evB["Level"] = ev["Level"]

            self.Events.append(evB)
            # print("    NodeB", evB["Start"].tt, evB["End"].tt)

            if ev["Parent"] is not None:
                ev["Parent"]["Child"].append(evB)

            self.add_maint_event(ts, evB)
            
                
        self.remove_maint(ev)
        
    def remove_maint(self, ev):
                
        for child in ev["Child"]:
            self.remove_maint(child)
        
        if ev["Parent"] is not None:
            ev["Parent"]["Child"].remove(ev)

        self.Events.remove(ev)
        
    def isBusy(self, ev1):
        
        if self.StorageCapacity < ev1["Ref"]['Storage']:
            return True
        
        for schedule in self.Scheduled:
            ev_begin = schedule[0]
            ev_end = schedule[1]
            
            if ev1["Start"].tt > ev_begin.tt and ev1["Start"].tt < ev_end.tt:
                return True

            if ev1["End"].tt > ev_begin.tt and ev1["End"].tt < ev_end.tt:
                return True

            if ev_begin.tt > ev1["Start"].tt and ev_begin.tt < ev1["End"].tt:
                return True

            if ev_end.tt > ev1["Start"].tt and ev_end.tt < ev1["End"].tt:
                return True
        
        #print("Getting Night Slots")
        ev_dur = ev1["Ref"]['Transfer Time']
        for slot in self.nightSlot["Slots"]:
            ev_begin = slot["Start"]
            ev_end = slot["End"]
            
            if ev1["Start"].tt > ev_begin.tt and ev1["Start"].tt < ev_end.tt:
                #print(slot, "slot + ev_dur", slot["Total Time"], ev_dur)
                if (slot["Total Time"] + ev_dur) > self.nightLimitMax:
                    #print("slot + ev_dur = Invalid")
                    return True

            if ev1["End"].tt > ev_begin.tt and ev1["End"].tt < ev_end.tt:
                #print(slot, "slot + ev_dur", slot["Total Time"], ev_dur)
                if (slot["Total Time"] + ev_dur) > self.nightLimitMax:
                    #print("slot + ev_dur = Invalid")
                    return True

            if ev_begin.tt > ev1["Start"].tt and ev_begin.tt < ev1["End"].tt:
                #print(slot, "slot + ev_dur", slot["Total Time"], ev_dur)
                if (slot["Total Time"] + ev_dur) > self.nightLimitMax:
                    #print("slot + ev_dur = Invalid")
                    return True

            if ev_end.tt > ev1["Start"].tt and ev_end.tt < ev1["End"].tt:
                #print(slot, "slot + ev_dur", slot["Total Time"], ev_dur)
                if (slot["Total Time"] + ev_dur) > self.nightLimitMax:
                    #print("slot + ev_dur = Invalid")
                    return True
            
        for outage in self.outages:
            ev_begin = outage["Start"]
            ev_end = outage["End"]
            
            if ev1["Start"].tt > ev_begin.tt and ev1["Start"].tt < ev_end.tt:
                return True

            if ev1["End"].tt > ev_begin.tt and ev1["End"].tt < ev_end.tt:
                return True

            if ev_begin.tt > ev1["Start"].tt and ev_begin.tt < ev1["End"].tt:
                return True

            if ev_end.tt > ev1["Start"].tt and ev_end.tt < ev1["End"].tt:
                return True

        return False

In [ ]:
class RLoptimizer:

    def __init__(self, sys):
        self.sys = sys
    
    def select(self, slots):
        
        bestSlot = slots[0]
        bestStationCongestion = len(slots[0]['Station'].allocatedSlots)
        bestSatCongestion = len(slots[0]['Sattelite'].Scheduled)

        #print("Slots for Image at", slots[0]['Event']['Ref']['Latitude'], slots[0]['Event']['Ref']['Longitude'])
        
        stations = []
        stationCongAvg = 0
        for slot in slots:
            if slot['Station'] not in stations:
                stations.append(slot['Station'])
                stationCongAvg += len(slot['Station'].allocatedSlots)
        
        if len(stations) > 0:
            stationCongAvg = stationCongAvg / len(stations)

        sats = []
        satCongAvg = 0
        for slot in slots:
            if slot['Sattelite'] not in sats:
                sats.append(slot['Sattelite'])
                satCongAvg += len(slot['Sattelite'].Scheduled)
            
        if len(sats) > 0:
            satCongAvg = satCongAvg / len(sats)
            
        bestScore = 999
        
        for slot in slots:
            #print("    Station", slot['Station'].Name, ", Sattelite", slot['Sattelite'].name, "at time", slot['Start'].utc_strftime())

            stationCongestion = len(slot['Station'].allocatedSlots)
            satCongestion = len(slot['Sattelite'].Scheduled)
            
            if satCongAvg != 0 and stationCongAvg!= 0:
                satScore = ((satCongestion-satCongAvg)/satCongAvg)**3
                stationScore = ((stationCongestion-stationCongAvg)/stationCongAvg)**3
                score = satScore + stationScore

                if score < bestScore:
                    bestScore = score
                    bestSlot = slot
            else:
                if stationCongestion > bestStationCongestion:
                    continue

                elif stationCongestion < bestStationCongestion:
                    bestStationCongestion = stationCongestion
                    bestSatCongestion = satCongestion
                    bestSlot = slot
                    continue

                elif satCongestion < bestSatCongestion:
                    bestStationCongestion = stationCongestion
                    bestSatCongestion = satCongestion
                    bestSlot = slot
                    continue
                        
        return bestSlot


In [ ]:
class system:

    def loadGS(self):
        
        self.GroundStations = []
        
        self.GroundStations.append(ground_station("Inuvik Northwest Territories", 68.3195, -133.549,
                                                  102.5, 5, 5, 40*1024, 100*1024*1024))
        
        self.GroundStations.append(ground_station("Prince Albert Saskatchewan", 53.2124, -105.934, 490.3,
                                                  5, 5, 40*1024, 100*1024*1024))
        
        self.GroundStations.append(ground_station("Gatineau Quebec", 45.5846, -75.8083, 240.1,
                                                  5, 5, 40*1024, 100*1024*1024))

    def canDownlink(self, ev, tm, sat):
        
        results = []
        
        for station in self.GroundStations:
            res = station.canDownlink(ev, tm, self.ts, sat.stationVisibility[station.Name]["slots"])
            
            if res[0]:
                res.append(station)
                
                results.append(res)
        
        #print("Can not uplink at time", tm.utc_strftime(), ev["Ref"]['DeliveryTime'], "from", sat.name)
        return results
        
    def loadOutages(self, batchName):
        outList = os.listdir(batchName + '/OutageRequests')
        
        for outFile in outList:
            with open(batchName + '/OutageRequests/'+outFile) as f:
                outDict = json.load(f)
                outDict["Start"] = self.strToTm(outDict["Window"]["Start"])
                outDict["End"] = self.strToTm(outDict["Window"]["End"])

                for sat in self.Satelites:
                    if sat.name == outDict['Target']:
                        sat.outages.append(outDict)
                        break

    def loadSat(self):

        self.Satelites = []

        satTLEs = os.listdir("data/satellite")

        for TLE in satTLEs:
            self.Satelites.append(satelite("data/satellite/"+TLE))


    def loadEPH(self):

        ## Step 2: (Maintenance) Is the satellite in eclipse or in sunlight?
        self.eph = load('data/de421.bsp')  # Load the JPL ephemeris DE421

    def loadOrders(self, batchName):
        orderList = os.listdir(batchName + '/ImageOrderRequests')

        ind = 0
        for orderFile in orderList:

            with open(batchName + '/ImageOrderRequests/'+orderFile) as f:
                ordDict = json.load(f)
                ordDict["Completed"] = False
                ordDict["Sources"] = {}
                ordDict["Schedule"] = None
                ordDict["Index"] = ind
                ordDict["Parent"] = None
                ordDict["Child"] = None
                ordDict["Processed"] = False
                                
                if ordDict['ImageType'] == 'Low':
                    ordDict['Length'] = 40
                    ordDict['Width'] = 20
                    ordDict['Transfer Time'] = 20
                    ordDict['Storage'] = 128*1024*1024
                    
                elif ordDict['ImageType'] == 'Medium':
                    ordDict['Length'] = 40
                    ordDict['Width'] = 20
                    ordDict['Transfer Time'] = 45
                    ordDict['Storage'] = 256*1024*1024
                    
                elif ordDict['ImageType'] == 'Spotlight' or ordDict['ImageType'] == 'High':
                    ordDict['Length'] = 10
                    ordDict['Width'] = 10
                    ordDict['Transfer Time'] = 120
                    ordDict['Storage'] = 512*1024*1024

                ordDict["ImageStartTimeObj"] = self.strToTm(ordDict["ImageStartTime"])
                ordDict["ImageEndTimeObj"] = self.strToTm(ordDict["ImageEndTime"])
                ordDict["Stage"] = batchName
                    
                self.orders.append(ordDict)

                ind = ind + 1
                
                dt1 = datetime.strptime(ordDict["ImageStartTime"], "%Y-%m-%dT%H:%M:%S")
                dt2 = datetime.strptime(ordDict["ImageEndTime"], "%Y-%m-%dT%H:%M:%S")
                
                windowDuration = (dt2-dt1).total_seconds()
                ordDict["CaptureWindow"] = windowDuration
                
                revisit = False
                if "Revisit" in ordDict["Recurrence"].keys():
                    revisit = ordDict["Recurrence"]["Revisit"]                    
                elif "Revist" in ordDict["Recurrence"].keys():
                    revisit = ordDict["Recurrence"]["Revist"]
                elif 'Revist' in ordDict["Recurrence"].keys():
                    revisit = ordDict["Recurrence"]['Revist']
                else:
                    print("Corrupt Order", ordDict["Recurrence"])
                
                self.optimizeOrder(ordDict)
                
                if revisit == "True":

                    parent = ordDict

                    freq = int(ordDict["Recurrence"]["RevisitFrequency"])
                    num = int(ordDict["Recurrence"]["NumberOfRevisits"])
                    
                    Days = 0
                    Hours = 0
                    
                    if ordDict["Recurrence"]["RevisitFrequencyUnits"] == "Days":
                        Days = freq
                        freq = freq*24*3600
                    elif ordDict["Recurrence"]["RevisitFrequencyUnits"] == "Hours":
                        Hours = freq
                        freq = freq*24
                    else:
                        print("Unknown Revisit Frequency Units:", ordDict["Recurrence"]["RevisitFrequencyUnits"])
                    
                    for i in range(num):
                        child = parent.copy()
                        
                        parent["Child"] = child
                        child["Parent"] = parent
                        child["Child"] = None
                        
                        dt = datetime.strptime(parent["ImageStartTime"], "%Y-%m-%dT%H:%M:%S")
                        dt = dt + timedelta(days = Days, hours = Hours)                        
                        child["ImageStartTime"] = dt.strftime("%Y-%m-%dT%H:%M:%S")

                        dt = datetime.strptime(parent["ImageEndTime"], "%Y-%m-%dT%H:%M:%S")
                        dt = dt + timedelta(days = Days, hours = Hours)                        
                        child["ImageEndTime"] = dt.strftime("%Y-%m-%dT%H:%M:%S")

                        dt = datetime.strptime(parent["DeliveryTime"], "%Y-%m-%dT%H:%M:%S")
                        dt = dt + timedelta(days = Days, hours = Hours)                        
                        child["DeliveryTime"] = dt.strftime("%Y-%m-%dT%H:%M:%S")
                        
                        child["ImageStartTimeObj"] = self.strToTm(child["ImageStartTime"])
                        child["ImageEndTimeObj"] = self.strToTm(child["ImageEndTime"])
                        
                        self.orders.append(child)
                        self.optimizeOrder(child)
                        
                        parent = child

            
        self.orders = sorted(self.orders, key=lambda d: d['CaptureWindow']) 
        print(len(self.orders))

    def loadMaint(self, batchName):
        maintList = os.listdir(batchName + '/MaintenanceRequests')

        for maintFile in maintList:
            
            with open(batchName + '/MaintenanceRequests/'+maintFile) as f:
                maintDict = json.load(f)
                maintDict['Completed'] = False

                if maintDict['PayloadOutage']:
                    for sat in self.Satelites:
                        if sat.name == maintDict['Target']:
                            sat.maint.append(maintDict)
                            break
        
    def __init__(self):

        self.ts = load.timescale() # Create timescale object for TLE computation
        self.loadGS()
        self.loadSat()
        self.loadEPH()

        self.orders = []
        
        self.optimizer = RLoptimizer(self)
        self.schedId = 0
        self.imageID = 0
        self.maintID = 0

    def strToTm(self, Str):        
        dt = datetime.strptime(Str, "%Y-%m-%dT%H:%M:%S")
        return self.ts.utc(dt.year, dt.month, dt.day, dt.hour, dt.minute, dt.second)
        

    def run(self, start, end, batchName):

        self.loadMaint(batchName)
        self.loadOutages(batchName)
        
        self.initOptimization()

        self.loadOrders(batchName)

        self.satelliteActivitySchedule()
        
    def ev_overlap(self, ev1, ev_begin, ev_end):        
        
        if ev1["Start"].tt > ev_begin.tt and ev1["Start"].tt < ev_end.tt:
            return True

        if ev1["End"].tt > ev_begin.tt and ev1["End"].tt < ev_end.tt:
            return True

        if ev_begin.tt > ev1["Start"].tt and ev_begin.tt < ev1["End"].tt:
            return True

        if ev_end.tt > ev1["Start"].tt and ev_end.tt < ev1["End"].tt:
            return True

        return False

    def bifurcate_maint_valid(self, ev, ev_begin, ev_end):
        
        if (ev_begin - ev["Start"]) > (int(ev["Ref"]['Duration']) + 5)/(24.0*3600.0):
            return True

        if (ev["End"] - ev_end) > (int(ev["Ref"]['Duration']) + 5)/(24.0*3600.0):
            return True

        #print("bifurcate_maint_valid", ev_begin.tt, ev_end.tt)
        #print("bifurcate_maint_valid", ev["Start"].tt, ev["End"].tt)
        
        return False

    def proc_image_event(self, ev, tm, sat):
        
        ev_begin = tm
        ev_dur = ev["Ref"]['Transfer Time']
        ev_end = self.ts.utc(tm.utc_datetime() + timedelta(seconds=ev_dur))
        
        valid = True
        overlap = False
        for maint in sat.maint:
            
            level = 0

            survive = False
            intersect=False
            for evM in sat.Events:
                if evM["Type"] != "Maintainence":
                    continue

                if evM["Ref"] != maint:
                    continue

                if int(evM["Level"]) != level:
                    continue

                if not self.ev_overlap(evM, ev_begin, ev_end):
                    continue

                #print("Overlap Order", evM["Level"], level)
                overlap = True
                intersect = True
                if self.bifurcate_maint_valid(evM, ev_begin, ev_end):
                    #print("Survive")
                    survive = True

            if intersect:
                if not survive:
                    valid = False
            
            if maint["RepeatCycle"]['Repetition'] != 'Null':
                for level in range(1, int(maint["RepeatCycle"]["Repetition"])):
                    
                    survive = False
                    intersect=False
                    for evM in sat.Events:
                        if evM["Type"] != "Maintainence":
                            continue

                        if evM["Ref"] != maint:
                            continue
                        
                        if int(evM["Level"]) != level:
                            continue
                            
                        if not self.ev_overlap(evM, ev_begin, ev_end):
                            continue

                        # print("Overlap Order", evM["Level"], level)
                        overlap = True
                        intersect = True
                        
                        if self.bifurcate_maint_valid(evM, ev_begin, ev_end):
                            #print("Survive")
                            survive = True
                            
                    if intersect:
                        if not survive:
                            valid = False

        return valid, overlap 
    
    def initOptimization(self):
        for sat in self.Satelites:
            for maintItem in sat.maint:
                # print(maintItem)
                ev = {}
                ev["Type"] = "Maintainence"
                ev["Start"] = self.strToTm(maintItem['Window']["Start"])
                ev["End"] = self.strToTm(maintItem['Window']["End"])
                ev["Ref"] = maintItem
                ev["Parent"] = None
                ev["Child"] = []
                ev["Level"] = 0
                
                sat.Events.append(ev)
                sat.add_maint_event(self.ts, ev)
                # print(ev["Start"], ev["End"])
    
    def getPossibleOrderSlots(self, order):
        slots = []
        consumed = False
        
        #print("Getting Possible Slots")
        tm = self.ts.utc(order["ImageStartTimeObj"].utc_datetime() + timedelta(seconds=-3600*6))
        end = self.ts.utc(order["ImageEndTimeObj"].utc_datetime() + timedelta(seconds=3600*6))
        while tm.tt < end.tt:  # Compare Julian dates
            
            for sat in self.Satelites:

                if sat.name not in order['Sources'].keys():
                    continue

                for ev in sat.Events:

                    if ev["Ref"] != order:
                        continue

                    if ev["Type"] != "Image Order":
                        continue

                    if (tm.tt - ev["Start"].tt) < 0 or (tm.tt - ev["End"].tt) > 0:
                        continue
                    
                    #print("Checking Busy")
                    if sat.isBusy(ev):
                        #print("Is Busy")
                        continue
                        
                    #print("Is Not Busy")
                
                    valid, overlap = self.proc_image_event(ev, tm, sat)

                    if not valid:
                        continue

                    #res, start, slot, station = self.canDownlink(ev, tm, sat)

                    results = self.canDownlink(ev, tm, sat)

                    if len(results) == 0:
                        continue
                    
                    for result in results:
                        res, start, slot, station = result
                        
                        if not res:
                            print("Corruption in results = self.canDownlink(ev, tm, sat)")
                            continue
                        
                        slotOrder = {}
                        slotOrder['Sattelite'] = sat
                        slotOrder['Station'] = station
                        slotOrder['Overlap'] = overlap
                        slotOrder['Start'] = start
                        slotOrder['Slot'] = slot
                        slotOrder['Time'] = tm
                        slotOrder['Event'] = ev
                        
                        #print("Ading Slot")
                        slots.append(slotOrder)
                        consumed = True

            tm = self.ts.utc(tm.utc_datetime() + timedelta(seconds=15)) # Print all variables every minute from start and end times.

        #print("Got Slots", slots)
        #print("Consumed", consumed)

        return consumed, slots
    
    def simulateOrder(self, order):

        start = self.ts.utc(order["ImageStartTimeObj"].utc_datetime() + timedelta(seconds=-3600*4))
        end = self.ts.utc(self.strToTm(order['DeliveryTime']).utc_datetime() + timedelta(seconds=3600*4))
                        
        for station in self.GroundStations:                            
            station.reset(start, end)
        
        tm = self.ts.utc(order["ImageStartTimeObj"].utc_datetime() + timedelta(seconds=-3600*4))
        end = self.ts.utc(order["ImageEndTimeObj"].utc_datetime() + timedelta(seconds=3600*4))
        while tm.tt < end.tt:  # Compare Julian dates
            for sat in self.Satelites:
                sat.update(tm, self.eph, [order], self.ts, self.GroundStations)

            tm = self.ts.utc(tm.utc_datetime() + timedelta(seconds=60)) # Print all variables every minute from start and end times.

        for sat in self.Satelites:
            sat.process_images_final([order], order["ImageEndTimeObj"])
                        
    def initOptimizationOrder(self, order):
        start = self.strToTm(order["ImageStartTime"])
        end = self.strToTm(order["ImageEndTime"])

        for sat in self.Satelites:

            #print(start, end)
            #print("-----------------------")

            if sat.name in order['Sources'].keys():
                for session in order['Sources'][sat.name]['sessions']:
                    #print(session)
                    if session[0].tt > start.tt and session[1].tt < end.tt:
                        ev = {}
                        ev["Type"] = "Image Order"
                        ev["Start"] = session[0]
                        ev["End"] = session[1]
                        ev["Ref"] = order

                        sat.Events.append(ev)
                        #print(ev["Start"], ev["End"])
                        
    def optimizeOrder(self, order):

        if order["Completed"]:
            return

        if order["Processed"]:
            return

        self.simulateOrder(order)
        self.initOptimizationOrder(order)
        
                    
        consumed, slots = self.getPossibleOrderSlots(order)
                
        if consumed:
            
            slotOrder = self.optimizer.select(slots)
            
            #print("Selected Slot", slotOrder)
            
            sat = slotOrder['Sattelite']
            station = slotOrder['Station']
            overlap = slotOrder['Overlap']
            start = slotOrder['Start']
            slot = slotOrder['Slot']
            tm = slotOrder['Time']
            ev = slotOrder['Event']
            station.bookSlot(ev, start, slot, self.ts)

            sat.register_event(tm, ev, self.ts)

            if overlap:

                ev_begin = tm
                ev_dur = ev["Ref"]['Transfer Time']
                ev_end = self.ts.utc(tm.utc_datetime() + timedelta(seconds=ev_dur))

                bfList = []
                for evM in sat.Events:
                    if evM["Type"] == "Maintainence":
                        if self.ev_overlap(evM, ev_begin, ev_end):
                            bfList.append(evM)

                for evM in bfList:                                
                    if evM in sat.Events:
                        sat.bifurcate_maint(evM, ev_begin, ev_end, self.ts)
                    else:                                    
                        validBif = False
                        for evP in bfList:                                        
                            if evP["Ref"] == evM["Ref"]:

                                evT = evM
                                while evT['Parent'] is not None:                                            
                                    if evP == evT['Parent']:
                                        validBif = True
                                        break

                                    evT = evT['Parent']

                        if not validBif:
                            print("Possible corruption")
            
            for sat in self.Satelites:

                if sat.name not in order['Sources'].keys():
                    continue

                evList = []
                for ev in sat.Events:
                    if ev["Type"] != "Image Order":
                        continue

                    if ev["Ref"] != order:
                        continue

                    evList.append(ev)

                for ev in evList:
                    sat.Events.remove(ev)
                    
        order["Processed"] = True

    def satelliteActivitySchedule(self):

        self.activitySchedules = []

        for sat in sys.Satelites:

            sched = {}
            sched["Satellite Name"] = sat.name
            sched["Schedule ID"] = self.schedId

            sched['Activity Window'] = {}
            sched['Activity Window']["Start"] = None
            sched['Activity Window']["End"] = None

            sched['Image Activities'] = []
            sched['Maintenance Activities'] = []
            sched['Downlink Activities'] = []

            for im in sat.Scheduled:

                if sched['Activity Window']["Start"] == None:
                    sched['Activity Window']["Start"] = im[0]
                elif sched['Activity Window']["Start"].tt > im[0].tt:
                    sched['Activity Window']["Start"] = im[0]

                if sched['Activity Window']["End"] == None:
                    sched['Activity Window']["End"] = im[2]['Ref']['Slot']['End']
                elif im[2]['Ref']['Slot']['End'].tt > sched['Activity Window']["End"].tt:
                    sched['Activity Window']["End"] = im[2]['Ref']['Slot']['End']

                imActivity = {}
                imActivity['Image ID'] = self.imageID
                imActivity['Type'] = im[2]['Ref']['ImageType']
                imActivity['Priority'] = im[2]['Ref']['Priority']
                imActivity['Image Time'] = im[0].utc_strftime("%Y-%m-%dT%H:%M:%S")

                sched['Image Activities'].append(imActivity)

                dnActivity = {}
                dnActivity['Image ID'] = self.imageID
                dnActivity['Downlink Start'] = im[2]['Ref']['Slot']['Start'].utc_strftime("%Y-%m-%dT%H:%M:%S")
                dnActivity['Downlink Stop'] = im[2]['Ref']['Slot']['End'].utc_strftime("%Y-%m-%dT%H:%M:%S")
                dnActivity['Station'] = im[2]['Ref']['Station']
                sched['Downlink Activities'].append(dnActivity)

                self.imageID = self.imageID + 1

            for maintItem in sat.maint:
                for evM in sat.Events:
                    if evM["Type"] != "Maintainence":
                        continue

                    if evM["Ref"] != maintItem:
                        continue

                    if int(evM["Level"]) != 0:
                        continue

                    event = evM
                    break

                mtActivity = {}
                mtActivity['Activity ID'] = self.maintID
                mtActivity['Description'] = event['Ref']['Activity']
                mtActivity['Activity Time'] = event['Start'].utc_strftime("%Y-%m-%dT%H:%M:%S")
                mtActivity['Payload Flag'] = event['Ref']['PayloadOutage']
                mtActivity['Duration'] = event['Ref']['Duration']
                sched['Maintenance Activities'].append(mtActivity)

                self.maintID = self.maintID + 1

                while len(event['Child']) > 0:
                    event = event['Child'][0]

                    mtActivity = {}
                    mtActivity['Activity ID'] = self.maintID
                    mtActivity['Description'] = event['Ref']['Activity']
                    mtActivity['Activity Time'] = event['Start'].utc_strftime("%Y-%m-%dT%H:%M:%S")
                    mtActivity['Payload Flag'] = event['Ref']['PayloadOutage']
                    mtActivity['Duration'] = event['Ref']['Duration']
                    sched['Maintenance Activities'].append(mtActivity)

                    self.maintID = self.maintID + 1

            if sched['Activity Window']["Start"] is not None:
                sched['Activity Window']["Start"] = sched['Activity Window']["Start"].utc_strftime("%Y-%m-%dT%H:%M:%S")
                sched['Activity Window']["End"] = sched['Activity Window']["End"].utc_strftime("%Y-%m-%dT%H:%M:%S")

                #print()
                #print()
                #print()
                #print(json.dumps(sched, indent=4))
                self.activitySchedules.append(sched)
                self.schedId = self.schedId + 1


In [ ]:
sys = system()

sys.run(start_time, end_time, 'batchData/Feed/Stage 1')
sys.run(start_time, end_time, 'batchData/Feed/Stage 2')
sys.run(start_time, end_time, 'batchData/Feed/Stage 3')
sys.run(start_time, end_time, 'batchData/Feed/Stage 4')
sys.run(start_time, end_time, 'batchData/Feed/Stage 5')
sys.run(start_time, end_time, 'batchData/Feed/Stage 6')
sys.run(start_time, end_time, 'batchData/Feed/Stage 7')


100
120
125
125
175
200
200


In [ ]:
if True:
    print()
    print()
    print("Image Failure Diagnostics:")
    print()

    print("Batch     Latitude          Longitude         Transfer   Slot      Station      Sattelite")
    print("                                              Time       Width     Visibility   Visibility")
    print("________________________________________________________________________________________________________________________")

    satTotal = len(sys.Satelites)
    for order in sys.orders:
        if not order["Completed"]:
            print(order["Stage"].split('/')[-1], end = " | ")
            print('{0:15.10f}'.format(order['Latitude']), end = " | ")
            print('{0:15.10f}'.format(order['Longitude']), end = " | ")

            transferTimeIssue = 0
            transferTimeTotal = 0

            slotWidthIssue = 0
            slotWidthTotal = 0

            visiblityIssue = 0
            visiblityTotal = 0

            satIssue = satTotal


            for sat in order['Sources']:

                if len(order['Sources'][sat]["sessions"]) > 0:
                    satIssue -= 1

                for slot in order['Sources'][sat]["sessions"]:

                    transferTimeTotal += 1                        
                    if order['Transfer Time'] > slot[2]:                            
                        transferTimeIssue += 1
                        transferTimeTotal += 1
                        continue

                    slotWidthTotal += 1
                    if sys.strToTm(order["ImageStartTime"]).tt > slot[0].tt:
                        print()
                        print(order["ImageStartTime"], slot[0].utc_strftime())

                        slotWidthIssue += 1
                        continue

                    if sys.strToTm(order["ImageEndTime"]).tt < slot[1].tt:
                        print()
                        print(order["ImageEndTime"], slot[1].utc_strftime())

                        slotWidthIssue += 1
                        continue

                    for satObj in sys.Satelites:
                        if satObj.name != sat:
                            continue

                        delivery = sys.strToTm(order["DeliveryTime"])
                        for station in satObj.stationVisibility:
                            for slotSt in satObj.stationVisibility[station]["slots"]:

                                #if slotSt["Start"].tt > slotSt["End"].tt:
                                #    continue

                                #print("Vis")

                                visiblityTotal += 1
                                if slotSt["Start"].tt > delivery.tt:
                                    visiblityIssue += 1
                                    continue

                                if slotSt["End"].tt < slot[0].tt:
                                    visiblityIssue += 1
                                    continue
                                
                                continue
                                print(sat, slot[0].utc_strftime(), delivery.utc_strftime())
                                print('{0: <10}'.format(order["ImageStartTime"]), end = " | ")
                                print('{0: <10}'.format(order["ImageEndTime"]), end = " | ")
                                #print('{0: <10}'.format(order['ImageType']), end = " | ")
                                #print('{0:15.10f}'.format(order['Latitude']), end = " | ")
                                #print('{0:15.10f}'.format(order['Longitude']), end = " | ")
                                #print('{0: <8}'.format(order['Priority']), end = " | ")
                                #print('{0:3.2f}'.format(order['CaptureWindow']/3600.0), end = "  ")

                                #print()                

                                #print("        ", slotSt["Start"].utc_strftime(), slotSt["End"].utc_strftime())


            transferPercent = 0
            if transferTimeTotal > 0:
                transferPercent = 100.0*transferTimeIssue/transferTimeTotal

            slotWidthPercent = 0
            if slotWidthTotal > 0:
                slotWidthPercent = 100.0*slotWidthIssue/slotWidthTotal

            visibilityPercent = 0
            if visiblityTotal > 0:
                visibilityPercent = 100.0*visiblityIssue/visiblityTotal

            satPercent = 100.0*satIssue/satTotal

            print('{0:6.2f}%'.format(transferPercent), end = "  | ")
            print('{0:6.2f}%'.format(slotWidthPercent), end = " | ")
            print('{0:6.2f}%'.format(visibilityPercent), end = "    | ")
            print('{0:6.2f}%'.format(satPercent), end = "    | ")
            print() 




Image Failure Diagnostics:

Batch     Latitude          Longitude         Transfer   Slot      Station      Sattelite
                                              Time       Width     Visibility   Visibility
________________________________________________________________________________________________________________________
Stage 5 |  -10.1382886060 |   39.2391341912 |   0.00%  |   0.00% |   0.00%    | 100.00%    | 
Stage 5 |  -10.1379057747 |   39.0081673629 |   0.00%  |   0.00% |   0.00%    | 100.00%    | 
Stage 5 |   -9.4258107816 |   37.4712029153 |   0.00%  |   0.00% |   0.00%    | 100.00%    | 
Stage 1 |   14.9954811746 |  120.4868359761 |   0.00%  |   0.00% |   0.00%    | 100.00%    | 
Stage 1 |   82.9030426486 |  -36.3605623850 |   0.00%  |   0.00% |   0.00%    | 100.00%    | 
Stage 1 |  -75.0150712352 |  124.9906699935 |   0.00%  |   0.00% |   0.00%    | 100.00%    | 
Stage 1 |   39.7253417301 |   43.8394252721 |   0.00%  |   0.00% |   0.00%    | 100.00%    | 
Stage 1 | 

In [ ]:
if True:
    notComplete = 0
    notPossible = 0
    satTotal = len(sys.Satelites)
    for order in sys.orders:
        if not order["Completed"]:
            notComplete += 1
            
            valid = False
            for sat in order['Sources']:

                if len(order['Sources'][sat]["sessions"]) > 0:
                    valid = True
                    break
            
            if not valid:
                notPossible = notPossible + 1
                
            
    print("Orders Not Completed =", notComplete, "/", len(sys.orders))
    print("Orders Completed =", len(sys.orders)-notComplete, "/", len(sys.orders))
    print("Orders Not Possible =", notPossible, "/", notComplete)


Orders Not Completed = 99 / 200
Orders Completed = 101 / 200
Orders Not Possible = 90 / 99


In [ ]:
if True:
    print()
    print()
    print("Image Available Resources:")
    print()

    print("Batch     Latitude          Longitude         Num sat.   Num GS    ")
    print("                                              available  available")
    print("________________________________________________________________________________________________________________________")


    satTotal = len(sys.Satelites)
    for order in sys.orders:

        print(order["Stage"].split('/')[-1], end = " | ")
        print('{0:15.10f}'.format(order['Latitude']), end = " | ")
        print('{0:15.10f}'.format(order['Longitude']), end = " | ")

        satName = set()
        gsName = set()

        for sat in order['Sources']:

            for slot in order['Sources'][sat]["sessions"]:

                if order['Transfer Time'] > slot[2]:                            
                    continue

                if sys.strToTm(order["ImageStartTime"]).tt > slot[0].tt:
                    continue

                if sys.strToTm(order["ImageEndTime"]).tt < slot[1].tt:
                    continue

                for satObj in sys.Satelites:
                    if satObj.name != sat:
                        continue

                    delivery = sys.strToTm(order["DeliveryTime"])
                    for station in satObj.stationVisibility:
                        for slotSt in satObj.stationVisibility[station]["slots"]:
                            if slotSt["Start"].tt > delivery.tt:
                                visiblityIssue += 1
                                continue

                            if slotSt["End"].tt < slot[0].tt:
                                visiblityIssue += 1
                                continue

                            satName.add(sat)
                            gsName.add(station)

        print('{0: <8}'.format(len(satName)), end = " | ")
        print('{0: <8}'.format(len(gsName)), end = " | ")
        print()



Image Available Resources:

Batch     Latitude          Longitude         Num sat.   Num GS    
                                              available  available
________________________________________________________________________________________________________________________
Stage 5 |  -10.1382886060 |   39.2391341912 | 0        | 0        | 
Stage 5 |  -10.1379057747 |   39.0081673629 | 0        | 0        | 
Stage 5 |   -9.4258107816 |   37.4712029153 | 0        | 0        | 
Stage 1 |   32.6848066387 |   76.2781155841 | 1        | 3        | 
Stage 1 |   14.9954811746 |  120.4868359761 | 0        | 0        | 
Stage 1 |   74.1066716132 |   96.5639073459 | 3        | 3        | 
Stage 1 |  -68.4280302140 |   92.3109097251 | 3        | 2        | 
Stage 1 |   82.9030426486 |  -36.3605623850 | 0        | 0        | 
Stage 1 |  -75.0150712352 |  124.9906699935 | 0        | 0        | 
Stage 1 |  -74.3992683326 |  -71.6100277589 | 3        | 3        | 
Stage 1 |   39.725341730

Stage 1 |   74.4526352793 |  108.3130677030 | 3        | 3        | 
Stage 1 |   37.4604556870 |   47.5084639237 | 0        | 0        | 
Stage 1 |  -79.2342272042 |   -5.7834574462 | 0        | 0        | 
Stage 2 |   23.1264426102 |   99.4737501012 | 1        | 3        | 
Stage 2 |   63.2375225681 |   16.3069006482 | 3        | 3        | 
Stage 2 |  -75.8830196067 |    8.8082493529 | 0        | 0        | 
Stage 1 |  -75.6424084612 |  137.8798135094 | 3        | 3        | 
Stage 1 |  -77.2218698904 |   70.2444216253 | 0        | 0        | 
Stage 2 |   21.1091051889 |  102.7472920090 | 0        | 0        | 
Stage 1 |   65.9867354357 |  176.3387460354 | 0        | 0        | 
Stage 1 |   67.6906587350 |  -49.1283371984 | 4        | 3        | 
Stage 1 |  -80.3235817013 |   -0.5961474132 | 5        | 3        | 
Stage 1 |   68.7172124221 |   22.8340885876 | 0        | 0        | 
Stage 5 |  -84.7950649808 |  -20.5374738287 | 0        | 0        | 
Stage 1 |    1.7036828717 |   29.4

In [ ]:
def genGroundStationRequest(self):
    for activity in self.activitySchedules:
        
        for sat in sys.Satelites:
            if sat.name == activity['Satellite Name']:
                break
                
        for station in sys.GroundStations:
            res, start, slot = station.canUplink(activity, self.start_sky, self.ts, sat.stationVisibility[station.Name]["slots"])
            
            if res:
                ev = {}
                ev["Ref"] = {}
                ev["Ref"]['Storage'] = len(json.dumps(activity))
                #station.bookSlot(ev, start, slot, self.ts)
                                
                gsRequest = {}
                gsRequest['Station Name'] = station.Name
                gsRequest['Satellite'] = sat.name
                gsRequest['Acquisition of Signal'] = None
                gsRequest['Loss of Signal'] = None
                
                for slot in sat.stationVisibility[station.Name]["slots"]:
                    if start.tt >= slot["Start"].tt and start.tt <= slot["End"].tt:
                        gsRequest['Acquisition of Signal'] = slot['Start'].utc_strftime("%Y-%m-%dT%H:%M:%S")
                        gsRequest['Loss of Signal'] = slot['End'].utc_strftime("%Y-%m-%dT%H:%M:%S")

                
                gsRequest['Satellite Schedule ID'] = activity["Schedule ID"]
                gsRequest['Images to be Downlinked'] = []
                
                print(gsRequest)
                
                break
            
            
#genGroundStationRequest(sys)

In [ ]:
if True:
    print("Image Capture Scedule:")
    print()
    print("Sat      Start Time                End Time                  Type         Latitude          Longitude         Priority")
    print("________________________________________________________________________________________________________________________")
    for order in sys.orders:
        if order["Completed"]:

            print(order["Sat"], end = " | ")
            print(order["Time Start"].utc_strftime(), end = " | ")
            print(order["Time End"].utc_strftime(), end = " | ")
            print('{0: <10}'.format(order['ImageType']), end = " | ")
            print('{0:15.10f}'.format(order['Latitude']), end = " | ")
            print('{0:15.10f}'.format(order['Longitude']), end = " | ")
            print('{0: <8}'.format(order['Priority']), end = "  ")

            print()

    print()
    print()
    print("Image Downlink Scedule:")
    print()
    print("Sat      Latitude          Longitude         Ground Station               Delivery Start            Delivery End")
    print("________________________________________________________________________________________________________________________")
    for order in sys.orders:
        if order["Completed"]:

            print(order["Sat"], end = " | ")
            print('{0:14.9f}'.format(order['Latitude']), end = " | ")
            print('{0:14.9f}'.format(order['Longitude']), end = " | ")
            print('{0: <28}'.format(order['Station']), end = " | ")
            print(order["Slot"]["Start"].utc_strftime(), end = " | ")
            print(order["Slot"]["End"].utc_strftime(), end = "")

            print()



Image Capture Scedule:

Sat      Start Time                End Time                  Type         Latitude          Longitude         Priority
________________________________________________________________________________________________________________________
SOSO-1 | 2023-10-02 05:05:56 UTC | 2023-10-02 05:06:41 UTC | Medium     |   32.6848066387 |   76.2781155841 | 1         
SOSO-4 | 2023-10-02 02:27:06 UTC | 2023-10-02 02:27:26 UTC | Low        |   74.1066716132 |   96.5639073459 | 3         
SOSO-3 | 2023-10-02 05:17:40 UTC | 2023-10-02 05:18:25 UTC | Medium     |  -68.4280302140 |   92.3109097251 | 2         
SOSO-1 | 2023-10-02 01:21:48 UTC | 2023-10-02 01:22:08 UTC | Low        |  -74.3992683326 |  -71.6100277589 | 2         
SOSO-3 | 2023-10-02 11:26:24 UTC | 2023-10-02 11:26:44 UTC | Low        |  -72.3005773638 |  135.5845378658 | 2         
SOSO-5 | 2023-10-02 09:46:15 UTC | 2023-10-02 09:47:00 UTC | Medium     |   81.0961329976 |  -40.4092929276 | 1         
SOSO-2 | 2

In [ ]:
if True:
    print()
    print()
    print("Maintainence Activity Scedule:")
    print()

    for sat in sys.Satelites:

        print()
        print()
        print("Sattelite:", sat.name)
        print()
        print("Activity                    Repetitions   Iteration   Start Time                End Time")
        print("________________________________________________________________________________________________________")
        for maintItem in sat.maint:
            #print(maintItem)
            #break

            print('{0: <25}'.format(maintItem['Activity']), end=' | ')

            print('{0: <11}'.format(maintItem["RepeatCycle"]['Repetition']), end = " | ")

            for evM in sat.Events:
                if evM["Type"] != "Maintainence":
                    continue

                if evM["Ref"] != maintItem:
                    continue

                if int(evM["Level"]) != 0:
                    continue

                event = evM

            ev_end = sys.ts.utc(event["Start"].utc_datetime() + timedelta(seconds=int(event["Ref"]["Duration"])))

            print('{0: <9}'.format(str(event["Level"])), end = " | " )
            print(event["Start"].utc_strftime(), end = " | ")
            print(ev_end.utc_strftime(), end = "  ")

            print()


            while len(event['Child']) > 0:
                event = event['Child'][0]

                ev_end = sys.ts.utc(event["Start"].utc_datetime() + timedelta(seconds=int(event["Ref"]["Duration"])))
                print("                          |             |", '{0: <9}'.format(str(event["Level"])), end = " | " )
                print(event["Start"].utc_strftime(), end = " | ")
                print(ev_end.utc_strftime(), end = "  ")

                print()
            print()




Maintainence Activity Scedule:



Sattelite: SOSO-1

Activity                    Repetitions   Iteration   Start Time                End Time
________________________________________________________________________________________________________
MemoryScrub               | 3           | 0         | 2023-10-02 00:00:00 UTC | 2023-10-02 00:03:00 UTC  
                          |             | 1         | 2023-10-03 16:03:00 UTC | 2023-10-03 16:06:00 UTC  
                          |             | 2         | 2023-10-05 08:06:00 UTC | 2023-10-05 08:09:00 UTC  

OrbitManeuver             | Null        | 0         | 2023-10-02 05:22:34 UTC | 2023-10-02 05:37:34 UTC  

OrbitParameterUpdate      | 4           | 0         | 2023-10-02 00:00:00 UTC | 2023-10-02 00:00:30 UTC  
                          |             | 1         | 2023-10-08 00:00:30 UTC | 2023-10-08 00:01:00 UTC  
                          |             | 2         | 2023-10-14 00:01:00 UTC | 2023-10-14 00:01:30 UTC  
       

In [ ]:
if True:
    print()
    print()
    print("Storage Usage:")
    print()
    print("Sattelite      Storage Left")
    print("__________________________")
    for sat in sys.Satelites:

        print(sat.name, "     ", end = " | ")

        print(sat.StorageCapacity/(1024.0*1024.0*1024.0), "GB")






Storage Usage:

Sattelite      Storage Left
__________________________
SOSO-1       | 36.375 GB
SOSO-2       | 36.625 GB
SOSO-3       | 35.875 GB
SOSO-4       | 35.875 GB
SOSO-5       | 36.375 GB


In [ ]:

print()
print()
print("Ground Station Allocation:")
print("Station                          Number of slots")
print("____________________________________________________")

for station in sys.GroundStations:
    
    print('{0: <30}'.format(station.Name), end = " | ")
    print(len(station.allocatedSlots))

    
print()
print()
print("Sattelite Allocation:")
print("Sattelite                        Number of slots")
print("____________________________________________________")
    
for sat in sys.Satelites:

    print('{0: <30}'.format(sat.name), end = " | ")
    print(len(sat.Scheduled))

#print()
#print()
#print("Ground Station Utilization:")
#print()

#for station in sys.GroundStations:

#    print()
#    print()
#    print(station.Name)

#    print()
#    print("Allocated Slots")
#    print("Start                       End")
#    print("____________________________________________________")
#    for slot in station.allocatedSlots:
#        print(slot["Start"].utc_strftime(), " | ", slot["End"].utc_strftime())
    
    
#    print()
#    print("Available Slots")
#    print("Start                       End")
#    print("____________________________________________________")
#    for slot in station.availableSlots:
#        print(slot["Start"].utc_strftime(), "   ", slot["End"].utc_strftime())




Ground Station Allocation:
Station                          Number of slots
____________________________________________________
Inuvik Northwest Territories   | 33
Prince Albert Saskatchewan     | 33
Gatineau Quebec                | 35


Sattelite Allocation:
Sattelite                        Number of slots
____________________________________________________
SOSO-1                         | 20
SOSO-2                         | 20
SOSO-3                         | 20
SOSO-4                         | 22
SOSO-5                         | 19


In [ ]:
print()
print()
print("Ground Station Visibility")
print()

for sat in sys.Satelites:
    print(sat.name)
    for station in sat.stationVisibility:
        print()
        print("    ", station)
        print("        ", "Start                       End")
        print("        ", "____________________________________________________")
        for slot in sat.stationVisibility[station]["slots"]:
            print("        ", slot["Start"].utc_strftime(), "   ", slot["End"].utc_strftime())




Ground Station Visibility

SOSO-1

     Inuvik Northwest Territories
         Start                       End
         ____________________________________________________
         2023-10-02 05:24:56 UTC     2023-10-02 05:26:26 UTC
         2023-10-02 06:56:26 UTC     2023-10-02 07:04:26 UTC
         2023-10-02 08:29:56 UTC     2023-10-02 08:39:26 UTC
         2023-10-02 05:25:03 UTC     2023-10-02 05:26:33 UTC
         2023-10-02 06:56:33 UTC     2023-10-02 07:04:33 UTC
         2023-10-02 16:13:49 UTC     2023-10-02 16:21:49 UTC
         2023-10-02 17:46:19 UTC     2023-10-02 17:55:19 UTC
         2023-10-02 19:20:19 UTC     2023-10-02 19:28:49 UTC
         2023-10-02 20:57:19 UTC     2023-10-02 21:01:49 UTC
         2023-10-02 05:25:06 UTC     2023-10-02 05:26:06 UTC
         2023-10-02 06:56:36 UTC     2023-10-02 07:04:36 UTC
         2023-10-01 21:22:04 UTC     2023-10-01 21:23:34 UTC
         2023-10-02 05:25:04 UTC     2023-10-02 05:26:04 UTC
         2023-10-01 21:22:29 UTC 

         2023-10-02 05:25:13 UTC     2023-10-02 05:26:13 UTC
         2023-10-02 06:56:43 UTC     2023-10-02 07:04:43 UTC
         2023-10-02 08:30:13 UTC     2023-10-02 08:39:13 UTC
         2023-10-02 10:03:43 UTC     2023-10-02 10:12:13 UTC
         2023-10-02 11:37:13 UTC     2023-10-02 11:44:13 UTC
         2023-10-02 13:10:13 UTC     2023-10-02 13:15:43 UTC
         2023-10-02 14:42:13 UTC     2023-10-02 14:48:13 UTC
         2023-10-02 05:25:11 UTC     2023-10-02 05:26:11 UTC
         2023-10-02 06:56:41 UTC     2023-10-02 07:04:41 UTC
         2023-10-02 05:25:06 UTC     2023-10-02 05:26:06 UTC
         2023-10-02 06:56:36 UTC     2023-10-02 07:04:36 UTC
         2023-10-02 08:30:06 UTC     2023-10-02 08:39:06 UTC
         2023-10-02 10:03:36 UTC     2023-10-02 10:12:06 UTC
         2023-10-02 11:37:06 UTC     2023-10-02 11:44:06 UTC
         2023-10-02 13:10:06 UTC     2023-10-02 13:15:36 UTC
         2023-10-02 14:42:06 UTC     2023-10-02 14:48:06 UTC
         2023-10-02 14:4

         2023-10-02 20:56:58 UTC     2023-10-02 21:01:28 UTC
         2023-10-02 17:46:09 UTC     2023-10-02 17:55:09 UTC
         2023-10-02 19:20:09 UTC     2023-10-02 19:28:39 UTC
         2023-10-02 20:57:09 UTC     2023-10-02 21:01:39 UTC
         2023-10-02 16:13:43 UTC     2023-10-02 16:21:43 UTC
         2023-10-02 17:46:13 UTC     2023-10-02 17:55:13 UTC
         2023-10-02 19:20:13 UTC     2023-10-02 19:28:43 UTC
         2023-10-02 20:57:13 UTC     2023-10-02 21:01:43 UTC
         2023-10-02 17:46:09 UTC     2023-10-02 17:55:09 UTC
         2023-10-02 19:20:09 UTC     2023-10-02 19:28:39 UTC
         2023-10-02 20:57:09 UTC     2023-10-02 21:01:39 UTC
         2023-10-02 16:13:52 UTC     2023-10-02 16:21:52 UTC
         2023-10-02 17:46:22 UTC     2023-10-02 17:55:22 UTC
         2023-10-02 19:20:22 UTC     2023-10-02 19:28:52 UTC
         2023-10-02 20:56:52 UTC     2023-10-02 21:01:52 UTC
         2023-10-01 21:22:10 UTC     2023-10-01 21:23:40 UTC
         2023-10-02 05:2

         2023-10-02 13:10:20 UTC     2023-10-02 13:15:50 UTC
         2023-10-02 14:41:50 UTC     2023-10-02 14:48:20 UTC
         2023-10-02 16:13:50 UTC     2023-10-02 16:21:50 UTC
         2023-10-02 17:46:20 UTC     2023-10-02 17:55:20 UTC
         2023-10-02 19:20:20 UTC     2023-10-02 19:28:50 UTC
         2023-10-02 20:56:50 UTC     2023-10-02 21:01:50 UTC
         2023-10-02 05:24:56 UTC     2023-10-02 05:26:26 UTC
         2023-10-02 06:56:26 UTC     2023-10-02 07:04:26 UTC
         2023-10-02 08:29:56 UTC     2023-10-02 08:39:26 UTC
         2023-10-02 10:03:56 UTC     2023-10-02 10:12:26 UTC
         2023-10-02 11:37:26 UTC     2023-10-02 11:43:56 UTC
         2023-10-02 13:10:26 UTC     2023-10-02 13:15:26 UTC
         2023-10-02 14:41:56 UTC     2023-10-02 14:48:26 UTC
         2023-10-02 16:13:26 UTC     2023-10-02 16:21:26 UTC
         2023-10-02 17:46:26 UTC     2023-10-02 17:55:26 UTC
         2023-10-02 19:20:26 UTC     2023-10-02 19:28:56 UTC
         2023-10-02 20:5

         2023-10-02 20:57:10 UTC     2023-10-02 21:01:40 UTC
         2023-10-02 05:25:06 UTC     2023-10-02 05:26:06 UTC
         2023-10-02 06:56:36 UTC     2023-10-02 07:04:36 UTC
         2023-10-02 08:30:06 UTC     2023-10-02 08:39:06 UTC
         2023-10-02 10:03:36 UTC     2023-10-02 10:12:06 UTC
         2023-10-02 11:37:06 UTC     2023-10-02 11:44:06 UTC
         2023-10-02 13:10:06 UTC     2023-10-02 13:15:36 UTC
         2023-10-02 14:42:06 UTC     2023-10-02 14:48:06 UTC
         2023-10-02 16:13:36 UTC     2023-10-02 16:21:36 UTC
         2023-10-02 17:46:06 UTC     2023-10-02 17:55:06 UTC
         2023-10-02 19:20:06 UTC     2023-10-02 19:29:06 UTC
         2023-10-02 20:57:06 UTC     2023-10-02 21:01:36 UTC
         2023-10-02 08:30:12 UTC     2023-10-02 08:39:12 UTC
         2023-10-02 10:03:42 UTC     2023-10-02 10:12:12 UTC
         2023-10-02 11:37:12 UTC     2023-10-02 11:44:12 UTC
         2023-10-02 13:10:12 UTC     2023-10-02 13:15:42 UTC
         2023-10-02 14:4

...

         2023-10-02 04:14:04 UTC     2023-10-02 04:15:04 UTC
         2023-10-02 05:45:04 UTC     2023-10-02 05:54:04 UTC
         2023-10-02 07:19:04 UTC     2023-10-02 07:27:04 UTC
         2023-10-02 05:45:07 UTC     2023-10-02 05:54:07 UTC
         2023-10-02 07:19:07 UTC     2023-10-02 07:27:07 UTC
         2023-10-02 05:44:45 UTC     2023-10-02 05:54:15 UTC
         2023-10-02 07:19:15 UTC     2023-10-02 07:26:45 UTC
         2023-10-02 04:14:13 UTC     2023-10-02 04:15:13 UTC
         2023-10-02 05:44:43 UTC     2023-10-02 05:54:13 UTC
         2023-10-02 07:19:13 UTC     2023-10-02 07:26:43 UTC
         2023-10-02 14:58:13 UTC     2023-10-02 15:02:43 UTC
         2023-10-02 04:14:11 UTC     2023-10-02 04:15:11 UTC
         2023-10-02 05:44:41 UTC     2023-10-02 05:54:11 UTC
         2023-10-02 07:19:11 UTC     2023-10-02 07:26:41 UTC
         2023-10-02 05:45:06 UTC     2023-10-02 05:54:06 UTC
         2023-10-02 07:19:06 UTC     2023-10-02 07:27:06 UTC
         2023-10-02 14:5

         2023-10-02 02:39:32 UTC     2023-10-02 02:45:02 UTC
         2023-10-02 04:12:02 UTC     2023-10-02 04:21:02 UTC
         2023-10-02 05:48:02 UTC     2023-10-02 05:50:32 UTC
         2023-10-02 13:22:02 UTC     2023-10-02 13:25:32 UTC
         2023-10-02 14:52:02 UTC     2023-10-02 15:01:02 UTC
         2023-10-02 04:18:41 UTC     2023-10-02 04:21:11 UTC
         2023-10-02 05:48:11 UTC     2023-10-02 05:50:41 UTC
         2023-10-02 13:21:41 UTC     2023-10-02 13:25:11 UTC
         2023-10-02 14:52:11 UTC     2023-10-02 15:01:11 UTC
         2023-10-02 16:28:41 UTC     2023-10-02 16:33:11 UTC
         2023-10-03 02:22:41 UTC     2023-10-03 02:24:11 UTC
         2023-10-02 05:48:10 UTC     2023-10-02 05:50:40 UTC
         2023-10-02 13:21:40 UTC     2023-10-02 13:25:10 UTC
         2023-10-02 14:52:10 UTC     2023-10-02 15:01:10 UTC
         2023-10-02 16:28:40 UTC     2023-10-02 16:33:10 UTC
         2023-10-02 02:39:10 UTC     2023-10-02 02:45:10 UTC
         2023-10-02 04:1

         2023-10-02 09:47:34 UTC     2023-10-02 09:56:04 UTC
         2023-10-02 11:21:34 UTC     2023-10-02 11:28:34 UTC
         2023-10-02 12:55:04 UTC     2023-10-02 13:01:04 UTC
         2023-10-02 06:39:07 UTC     2023-10-02 06:47:07 UTC
         2023-10-02 08:13:07 UTC     2023-10-02 08:22:37 UTC
         2023-10-02 09:47:07 UTC     2023-10-02 09:56:07 UTC
         2023-10-02 11:21:37 UTC     2023-10-02 11:28:37 UTC
         2023-10-02 12:55:07 UTC     2023-10-02 13:01:07 UTC
         2023-10-02 06:39:15 UTC     2023-10-02 06:46:45 UTC
         2023-10-02 08:13:15 UTC     2023-10-02 08:22:45 UTC
         2023-10-02 09:47:15 UTC     2023-10-02 09:56:15 UTC
         2023-10-02 11:21:15 UTC     2023-10-02 11:28:45 UTC
         2023-10-02 12:55:15 UTC     2023-10-02 13:00:45 UTC
         2023-10-02 06:39:13 UTC     2023-10-02 06:46:43 UTC
         2023-10-02 08:13:13 UTC     2023-10-02 08:22:43 UTC
         2023-10-02 09:47:13 UTC     2023-10-02 09:56:13 UTC
         2023-10-02 11:2

         2023-10-02 06:38:55 UTC     2023-10-02 06:46:55 UTC
         2023-10-02 08:12:55 UTC     2023-10-02 08:22:25 UTC
         2023-10-02 09:47:25 UTC     2023-10-02 09:56:25 UTC
         2023-10-02 11:21:25 UTC     2023-10-02 11:28:55 UTC
         2023-10-02 12:54:55 UTC     2023-10-02 13:00:55 UTC
         2023-10-02 14:27:25 UTC     2023-10-02 14:33:55 UTC
         2023-10-02 15:59:55 UTC     2023-10-02 16:07:55 UTC
         2023-10-02 17:32:55 UTC     2023-10-02 17:41:55 UTC
         2023-10-02 19:07:25 UTC     2023-10-02 19:16:25 UTC
         2023-10-02 06:38:57 UTC     2023-10-02 06:46:57 UTC
         2023-10-02 08:12:57 UTC     2023-10-02 08:22:27 UTC
         2023-10-02 09:47:27 UTC     2023-10-02 09:56:27 UTC
         2023-10-02 11:21:27 UTC     2023-10-02 11:28:57 UTC
         2023-10-02 12:54:57 UTC     2023-10-02 13:00:57 UTC
         2023-10-02 14:27:27 UTC     2023-10-02 14:33:57 UTC
         2023-10-02 15:59:57 UTC     2023-10-02 16:07:57 UTC
         2023-10-02 17:3

         2023-10-02 15:51:09 UTC     2023-10-02 15:59:39 UTC
         2023-10-02 03:34:07 UTC     2023-10-02 03:43:37 UTC
         2023-10-02 05:09:07 UTC     2023-10-02 05:16:07 UTC
         2023-10-02 03:34:27 UTC     2023-10-02 03:43:27 UTC
         2023-10-02 05:08:57 UTC     2023-10-02 05:16:27 UTC
         2023-10-02 14:16:48 UTC     2023-10-02 14:25:48 UTC
         2023-10-02 15:51:18 UTC     2023-10-02 15:59:48 UTC
         2023-10-02 14:17:10 UTC     2023-10-02 14:25:40 UTC
         2023-10-02 15:51:10 UTC     2023-10-02 15:59:40 UTC
         2023-10-02 14:17:08 UTC     2023-10-02 14:25:38 UTC
         2023-10-02 15:51:08 UTC     2023-10-02 15:59:38 UTC
         2023-10-02 05:09:00 UTC     2023-10-02 05:16:30 UTC
         2023-10-02 03:34:32 UTC     2023-10-02 03:43:32 UTC
         2023-10-02 05:09:02 UTC     2023-10-02 05:16:32 UTC
         2023-10-02 14:17:02 UTC     2023-10-02 14:25:32 UTC
         2023-10-02 15:51:32 UTC     2023-10-02 15:59:32 UTC
         2023-10-02 14:1

In [ ]:
print()
print()
print("Ground Station Day Night")
print()

for sat in sys.Satelites:
    print(sat.name)
    
    for slot in sat.nightSlot["Slots"]:
        print("    ", slot["Start"].utc_strftime(), slot["End"].utc_strftime(), slot["Total Time"])
    
    
    
    
    



Ground Station Day Night

SOSO-1
     2023-10-02 00:41:56 UTC 2023-10-02 01:16:56 UTC 0
     2023-10-02 02:15:56 UTC 2023-10-02 02:50:56 UTC 20
     2023-10-02 03:50:56 UTC 2023-10-02 04:25:56 UTC 0
     2023-10-02 05:24:56 UTC 2023-10-02 05:59:56 UTC 0
     2023-10-02 06:59:56 UTC 2023-10-02 07:34:56 UTC 0
     2023-10-02 08:33:56 UTC 2023-10-02 09:08:56 UTC 45
     2023-10-01 23:07:03 UTC 2023-10-01 23:42:03 UTC 0
     2023-10-02 15:22:49 UTC 2023-10-02 15:26:49 UTC 0
     2023-10-02 16:25:49 UTC 2023-10-02 17:00:49 UTC 0
     2023-10-02 18:00:49 UTC 2023-10-02 18:35:49 UTC 0
     2023-10-02 19:34:49 UTC 2023-10-02 20:09:49 UTC 45
     2023-10-02 21:09:49 UTC 2023-10-02 21:44:49 UTC 65
     2023-10-02 22:43:49 UTC 2023-10-02 23:18:49 UTC 20
     2023-10-03 00:17:49 UTC 2023-10-03 00:53:49 UTC 0
     2023-10-03 01:52:49 UTC 2023-10-03 02:27:49 UTC 0
     2023-10-03 03:26:49 UTC 2023-10-01 20:51:04 UTC 0
     2023-10-01 21:33:04 UTC 2023-10-01 22:08:04 UTC 0
     2023-10-01 20:27:59 